In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [ ]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
cyber_data = load_dataset('bhattbhavesh91/cyber-dataset')

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/48.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93664 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/31218 [00:00<?, ? examples/s]

In [ ]:
cyber_data

DatasetDict({
    train: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels'],
        num_rows: 93664
    })
    test: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels'],
        num_rows: 31218
    })
})

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["crimeaditionalinfo"],
        padding=True,
        truncation=True,
        max_length=512
    )

In [ ]:
cyber_data_encoded = cyber_data.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/93664 [00:00<?, ? examples/s]

Map:   0%|          | 0/31218 [00:00<?, ? examples/s]

In [ ]:
cyber_data_encoded

DatasetDict({
    train: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 93664
    })
    test: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31218
    })
})

In [ ]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

cyber_data_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 32

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(cyber_data_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(cyber_data_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[  101,  9768,  2909, ...,     0,     0,     0],
       [  101,  2016,  1998, ...,     0,     0,     0],
       [  101,  2204,  5027, ...,     0,     0,     0],
       ...,
       [  101,  2171, 18529, ...,     0,     0,     0],
       [  101,  2909,  6583, ...,     0,     0,     0],
       [  101,  2909,  2023, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([3 0 9 4 1 7 1 0 1 0 3 6 

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
class SigmoidFocalCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0, from_logits=False, reduction=tf.keras.losses.Reduction.NONE, name='sigmoid_focal_crossentropy'):
        """
        Implementation of Sigmoid Focal Cross Entropy loss.

        Args:
            alpha: Weight factor to address class imbalance (default: 0.25)
            gamma: Focusing parameter to reduce the loss contribution from easy examples (default: 2.0)
            from_logits: Whether the predictions are logits or probabilities (default: False)
            reduction: Type of reduction to apply to the loss
            name: Name of the loss function
        """
        super().__init__(reduction=reduction, name=name)
        self.alpha = alpha
        self.gamma = gamma
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        """
        Calculate focal loss between ground truth and predictions.

        Args:
            y_true: Ground truth labels (sparse format)
            y_pred: Model predictions

        Returns:
            Calculated focal loss
        """
        # Convert sparse labels to one-hot format
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])

        # Apply sigmoid if inputs are logits
        if self.from_logits:
            y_pred = tf.nn.softmax(y_pred)

        # Clip predictions to prevent numerical instability
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Calculate cross entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate focal weight
        p_t = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
        alpha_factor = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        modulating_factor = tf.pow((1.0 - p_t), self.gamma)

        # Calculate focal loss
        focal_loss = alpha_factor * modulating_factor * cross_entropy

        # Sum over classes
        focal_loss = tf.reduce_sum(focal_loss, axis=-1)

        return focal_loss

In [ ]:
classifier = BERTForClassification(model, num_classes=14)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    loss=SigmoidFocalCrossEntropy(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
2927/2927 [==============================] - 1266s 419ms/step - loss: 0.7687 - accuracy: 0.7467
Epoch 2/3
2927/2927 [==============================] - 1224s 418ms/step - loss: 0.6471 - accuracy: 0.7737
Epoch 3/3
2927/2927 [==============================] - 1225s 418ms/step - loss: 0.5946 - accuracy: 0.7914


In [ ]:
classifier.evaluate(test_dataset)

976/976 [==============================] - 146s 146ms/step - loss: 0.6533 - accuracy: 0.7689


[0.6533125638961792, 0.7689153552055359]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("bhattbhavesh91/cyber-query-classifier-v2")

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]